# Imports

In [1]:
!pip install -q \
    torch \
    torchvision \
    torchaudio \
    transformers \
    datasets \
    nltk \
    faiss-cpu \
    matplotlib \
    tqdm \
    numpy \
    pandas \
    accelerate\
    sentencepiece


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
import os
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from typing import Optional
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from nltk.translate.bleu_score import sentence_bleu
import nltk
import faiss
import torch.nn.functional as F
from torch.utils.data import Dataset

import os
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from tqdm import tqdm

import itertools
import time
import warnings

# Chargement des données

In [4]:
from datasets import load_dataset
questions = load_dataset("rag-datasets/rag-mini-wikipedia", "question-answer")
text = load_dataset("rag-datasets/rag-mini-wikipedia", "text-corpus")

In [5]:
print(questions)
print(text)

DatasetDict({
    test: Dataset({
        features: ['question', 'answer', 'id'],
        num_rows: 918
    })
})
DatasetDict({
    passages: Dataset({
        features: ['passage', 'id'],
        num_rows: 3200
    })
})


In [6]:
from datasets import DatasetDict

questions_all = questions["test"]

# split = questions_all.train_test_split(test_size=0.2, seed=42)
# questions_train = split["train"]
# temp = split["test"].train_test_split(test_size=0.5, seed=42)
# questions_val = temp["train"]
# questions_test = temp["test"]

# questions_split = DatasetDict({
#     "train": questions_train,
#     "validation": questions_val,
#     "test": questions_test
# })

questions_val = questions_all
questions_split = DatasetDict({
    "validation": questions_val
})

In [7]:
print(questions_split)

DatasetDict({
    validation: Dataset({
        features: ['question', 'answer', 'id'],
        num_rows: 918
    })
})


## Constantes

In [8]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 16

## Encodeur

### Chargement de l'encodeur

In [9]:
from transformers import AutoTokenizer, AutoModel
model_name = 'BAAI/bge-base-en-v1.5'
emb_tokenizer = AutoTokenizer.from_pretrained(model_name)
emb_model = AutoModel.from_pretrained(model_name)
emb_model.to(DEVICE)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

### Stockage dans FAISS

In [10]:
def encode_sequences(sequences: list, tokenizer, model, device=DEVICE, batch_size=BATCH_SIZE, training=False) -> torch.Tensor:
    model.train() if training else model.eval()
    context = torch.enable_grad() if training else torch.no_grad()

    embeddings = []
    with context:
        for i in range(0, len(sequences), batch_size):
            batch = sequences[i:i + batch_size]

            # Tokenisation des textes avant entrée du modèle
            inputs = tokenizer(
                batch, padding=True, truncation=True, return_tensors="pt"
            ).to(device)

            # Passage dans le modèle encodeur
            output = model(**inputs).last_hidden_state

            # Pooling : moyenne des vecteurs de tous les tokens pour chaque séquence
            pooled = output.mean(dim=1)  # [batch_size, hidden_size]

            embeddings.append(pooled)

    # Concatène tous les embeddings pour obtenir un seul tensor de sortie
    return torch.cat(embeddings, dim=0)

In [11]:
def retrieve_top_k_passages_faiss(questions: list, vector_index: faiss.IndexFlatL2, embedding_model_tokenizer, embedding_model, k: int):
    question_embeddings = encode_sequences(questions, embedding_model_tokenizer, embedding_model)
    question_embeddings = question_embeddings.cpu().numpy()
    faiss.normalize_L2(question_embeddings)
    _, top_k_indices = vector_index.search(question_embeddings, k)
    
    return top_k_indices

In [12]:
passages = text["passages"]["passage"]
passage_embed = encode_sequences(passages, emb_tokenizer, emb_model)
passage_embeddings = passage_embed.cpu().numpy()
faiss.normalize_L2(passage_embeddings)

d = passage_embeddings.shape[1]
index = faiss.IndexFlatIP(d)
index.add(passage_embeddings)

## RAG Fusion

### Rephrasing

In [13]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "google/flan-t5-base"
rag_fus_tokenizer = AutoTokenizer.from_pretrained(model_name)
rag_fus_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16)
rag_fus_model.to(DEVICE)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [13]:
def generate_reformulations_flant5(question, model, tokenizer, n=5):
    prompt_template = (
        "Instruction: Rephrase the question in a totally different way\n\n"
        f"Input Question: {question.strip()}\n"
        "Rephrased Question:"
    )
    return [
        tokenizer.decode(
            model.generate(
                **tokenizer(prompt_template, return_tensors="pt").to(model.device),
                max_new_tokens=64,
                do_sample=True,
                top_k=50,
                top_p=0.95,
                temperature=0.9,
            )[0],
            skip_special_tokens=True,
        )
        for _ in range(n)
    ]

In [14]:
from transformers import T5ForConditionalGeneration,T5Tokenizer

rag_fus_model2 = T5ForConditionalGeneration.from_pretrained('ramsrigouthamg/t5_paraphraser')
rag_fus_tokenizer2 = T5Tokenizer.from_pretrained('ramsrigouthamg/t5_paraphraser')
rag_fus_model2 = rag_fus_model2.to(DEVICE)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [15]:
def generate_reformulations_t5paraphraser(question, model, tokenizer, n=5):
    text = f"paraphrase: {question} </s>"
    encoding = tokenizer.encode_plus(
        text, return_tensors="pt", padding=True, truncation=True, max_length=256
    ).to(model.device)

    print(text)

    outputs = model.generate(
        input_ids=encoding["input_ids"],
        attention_mask=encoding["attention_mask"],
        do_sample=True,
        max_length=256,
        top_k=120,
        top_p=0.98,
        early_stopping=True,
        num_return_sequences=n * 2  # générer plus et filtrer les doublons
    )

    results = []
    for output in outputs:
        decoded = tokenizer.decode(output, skip_special_tokens=True)
        if decoded.lower() != question.lower() and decoded not in results:
            results.append(decoded)
        if len(results) >= n:
            break
    return results

In [16]:
def generate_reformulations(question, model, tokenizer, method="flant5", n=5):
    if method == "flant5":
        return generate_reformulations_flant5(question, model, tokenizer, n)
    elif method == "paraphraser":
        return generate_reformulations_t5paraphraser(question, model, tokenizer, n)

In [17]:
from collections import defaultdict

def retrieve_rag_fusion_rrf(question, model, tokenizer, faiss_index, emb_model, emb_tokenizer, 
                            k=5, reform_method="flant5", n_reform=5, rrf_k=60, prefilter_top_n=20):
    reformulations = generate_reformulations(question, model, tokenizer, reform_method, n_reform)
    # print(reformulations)

    # RRF fusion
    rrf_scores = defaultdict(float)
    for q in reformulations:
        top_ids = retrieve_top_k_passages_faiss([q], faiss_index, emb_tokenizer, emb_model, k)[0]
        for rank, passage_id in enumerate(top_ids):
            score = 1 / (rank + 1 + rrf_k)
            rrf_scores[passage_id] += score

    # Trier et retourner les top-N documents
    sorted_ids = [doc_id for doc_id, _ in sorted(rrf_scores.items(), key=lambda x: x[1], reverse=True)]
    return sorted_ids[:prefilter_top_n]


def run_rag_fusion_pipeline_rrf(questions, reform_model, reform_tokenizer, reform_method, 
                                faiss_index, emb_model, emb_tokenizer, 
                                k=5, n_reform=5, rrf_k=60, prefilter_top_n=20):
    results = {}
    for q_index, question in enumerate(questions):
        print(f"[{q_index+1}/{len(questions)}] Processing question: {question}")
        
        retrieved_ids = retrieve_rag_fusion_rrf(
            question,
            reform_model,
            reform_tokenizer,
            faiss_index,
            emb_model,
            emb_tokenizer,
            k=k,
            reform_method=reform_method,
            n_reform=n_reform,
            rrf_k=rrf_k,
            prefilter_top_n=prefilter_top_n
        )

        results[question] = retrieved_ids
    return results

In [18]:
reform_method = "paraphraser"
# reform_method = "flant5"
questions_v = questions_split["validation"]["question"]

results = run_rag_fusion_pipeline_rrf(
    questions=questions_v,            
    reform_model=rag_fus_model2,
    reform_tokenizer=rag_fus_tokenizer2,
    reform_method=reform_method,
    faiss_index=index,
    emb_model=emb_model,
    emb_tokenizer=emb_tokenizer,
    k=5,
    n_reform=5,
    rrf_k=60,
    prefilter_top_n=30   # combien de passages on rerank après RRF
)

[1/918] Processing question: Was Abraham Lincoln the sixteenth President of the United States?
paraphrase: Was Abraham Lincoln the sixteenth President of the United States? </s>


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5.py:289: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:638: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


[2/918] Processing question: Did Lincoln sign the National Banking Act of 1863?
paraphrase: Did Lincoln sign the National Banking Act of 1863? </s>
[3/918] Processing question: Did his mother die of pneumonia?
paraphrase: Did his mother die of pneumonia? </s>
[4/918] Processing question: How many long was Lincoln's formal education?
paraphrase: How many long was Lincoln's formal education? </s>
[5/918] Processing question: When did Lincoln begin his political career?
paraphrase: When did Lincoln begin his political career? </s>
[6/918] Processing question: What did The Legal Tender Act of 1862 establish?
paraphrase: What did The Legal Tender Act of 1862 establish? </s>
[7/918] Processing question: Who suggested Lincoln grow a beard?
paraphrase: Who suggested Lincoln grow a beard? </s>
[8/918] Processing question: When did the Gettysburg address argue that America was born?
paraphrase: When did the Gettysburg address argue that America was born? </s>
[9/918] Processing question: Did Lin

In [19]:
print(results)

{'Was Abraham Lincoln the sixteenth President of the United States?': [278, 319, 367, 473, 388, 288, 198, 281, 635, 300, 387, 341, 352], 'Did Lincoln sign the National Banking Act of 1863?': [360, 1816, 278, 2155, 330, 2970, 325, 327, 352, 580], 'Did his mother die of pneumonia?': [2893, 1396, 2521, 1395, 2992, 698, 965, 285], "How many long was Lincoln's formal education?": [287, 278, 285, 289, 286, 376, 288, 296], 'When did Lincoln begin his political career?': [289, 278, 292, 352, 318, 300, 314, 310, 477], 'What did The Legal Tender Act of 1862 establish?': [361, 1816, 1819, 360, 2155], 'Who suggested Lincoln grow a beard?': [382, 287, 278, 289, 288, 334, 285, 375, 282], 'When did the Gettysburg address argue that America was born?': [358, 355, 282, 638, 278, 1767, 402, 380], 'Did Lincoln beat John C. Breckinridge in the 1860 election?': [319, 317, 318, 278, 335, 340, 178, 408, 450, 302, 177, 156, 300], 'Was Abraham Lincoln the first President of the United States?': [278, 319, 635,

## Generator

### Chargement du générateur

In [ ]:
from huggingface_hub import login
login(token = '')

In [21]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "mistralai/Mistral-7B-Instruct-v0.1"

gen_tokenizer = AutoTokenizer.from_pretrained(model_name)
gen_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",           # auto-placement sur GPU si dispo
    torch_dtype=torch.float16,   # allège la mémoire (fp16)
    trust_remote_code=True,       # pour activer les modèles qui utilisent du code perso
)
gen_tokenizer.pad_token = gen_tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Prompts

In [22]:
def generate_prompts_from_rag_fusion_results(questions, fusion_results, passages, top_k=5):
    prompts = []
    help_instruction = "Give an answer to the question. Use one word if possible."

    for i, question in enumerate(questions):
        passage_ids = fusion_results[question][:top_k]
        retrieved_passages = [passages[j] for j in passage_ids]

        context_string = (
            f"{help_instruction}\n" +
            "\n".join(retrieved_passages) + "\n" +
            f"Question: {question}\n"
        )
        prompts.append(context_string)

    return prompts

In [23]:
questions_val = questions_split["validation"]["question"]
passages = text["passages"]["passage"]

prompts_rag_fusion = generate_prompts_from_rag_fusion_results(
    questions=questions_val,
    fusion_results=results,
    passages=passages,
    top_k=3
)

# Affichage exemple
print(prompts_rag_fusion[0])

Give an answer to the question. Use one word if possible.
Abraham Lincoln (February 12, 1809 â April 15, 1865) was the sixteenth President of the United States, serving from March 4, 1861 until his assassination. As an outspoken opponent of the expansion of slavery in the United States, "[I]n his short autobiography written for the 1860 presidential campaign, Lincoln would describe his protest in the Illinois legislature as one that 'briefly defined his position on the slavery question, and so far as it goes, it was then the same that it is now." This was in reference to the anti-expansion sentiments he had then expressed. Doris Kearns Goodwin, Team of Rivals: The Political Genius of Abraham Lincoln (2005) p. 91.  Holzer pg. 232.  Writing of the Cooper Union  speech, Holzer notes, "Cooper Union proved a unique confluence of political culture, rhetorical opportunity, technological innovation, and human genius, and it brought Abraham Lincoln to the center stage of American politics at 

### Génération

In [24]:
def process_batch_prompts(batch_prompts, tokenizer, model, device, max_new_tokens=64):
    formatted_prompts = [
        f"<s>[INST] {prompt.strip()} [/INST]" for prompt in batch_prompts
    ]

    print(formatted_prompts)

    inputs = tokenizer(formatted_prompts, return_tensors="pt", padding=True, truncation=True).to(device)

    # Génération
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=max_new_tokens,
        do_sample=True,
        top_p=0.9,
        temperature=0.7,
        eos_token_id=tokenizer.eos_token_id
    )

    # Décodage et extraction de la réponse
    responses = [
        tokenizer.decode(output, skip_special_tokens=True).split('[/INST]')[-1].strip()
        for output in outputs
    ]

    print(responses)

    return responses

def chunked_iterable(iterable, size):
    it = iter(iterable)
    while True:
        chunk = tuple(itertools.islice(it, size))
        if not chunk:
            return
        yield chunk

def generate_responses(gen_model, gen_tokenizer, prompts, batch_size = 32) : 
    batch_responses = []
    
    for batch_index, batch_prompts in enumerate(chunked_iterable(prompts, batch_size)):
        print(f"Batch {batch_index}] Started generation...")
        start_time = time.time()
        
        batch_responses.extend(process_batch_prompts(batch_prompts, gen_tokenizer, gen_model, DEVICE))
        
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f"Batch {batch_index}] Time for generation = {elapsed_time:.2f} seconds")
    return batch_responses

In [25]:
generated_responses_fusion = generate_responses(gen_model, gen_tokenizer, prompts_rag_fusion)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Batch 0] Started generation...
['<s>[INST] Give an answer to the question. Use one word if possible.\nAbraham Lincoln (February 12, 1809 â\x80\x93 April 15, 1865) was the sixteenth President of the United States, serving from March 4, 1861 until his assassination. As an outspoken opponent of the expansion of slavery in the United States, "[I]n his short autobiography written for the 1860 presidential campaign, Lincoln would describe his protest in the Illinois legislature as one that \'briefly defined his position on the slavery question, and so far as it goes, it was then the same that it is now." This was in reference to the anti-expansion sentiments he had then expressed. Doris Kearns Goodwin, Team of Rivals: The Political Genius of Abraham Lincoln (2005) p. 91.  Holzer pg. 232.  Writing of the Cooper Union  speech, Holzer notes, "Cooper Union proved a unique confluence of political culture, rhetorical opportunity, technological innovation, and human genius, and it brought Abraham L

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


['Yes.', 'Yes.', 'No.', '18 months.', '1832.', 'Paper currency.', 'Grace Bedell', '1776', 'Yes.', 'No, Abraham Lincoln was not the first President of the United States. John Adams was the second President of the United States, serving from 1797 to 1801.', 'Yes.', 'Yes.', 'Hardin (now LaRue) County, Kentucky.', 'Lincoln served as President from March 4, 1861 until his assassination on April 15, 1865.', 'Booth', 'Yes.', 'McClellan', 'To free slaves in Confederate territories.', 'Yes.', 'Yes, Lincoln had 18 months of schooling.', 'Yes', 'Lincoln was seven years old in 1816.', '1846.', '23 years.', 'Courtroom.', 'Yes.', 'No.', 'Avogadro', 'John Adams', 'Never.', '1669', 'Yes.']
Batch 0] Time for generation = 6.76 seconds
Batch 1] Started generation...
["<s>[INST] Give an answer to the question. Use one word if possible.\nLorenzo Romano Amedeo Carlo Avogadro, Count of Quaregna and Cerreto (August 9, 1776 July 9, 1856) was an Italian savant.  He is most noted for his contributions to the the

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

['Yes.', 'Yes.', 'No.', 'Three.', 'Turtle.', '1. Turtle Care Guide\n2. Types of Turtles\n3. Turtle Diet\n4. Turtle Habitat', 'A female turtle.', 'Papillae.', 'Reference.', 'No.', 'Yes.', 'Yes.', 'Paracryptodira.', 'Yolk.', 'Beaches.', 'Pleurodira.', 'Sand or mud.', 'Reptiles.', 'Mud or sand.', 'All turtles are ectothermic.', 'No.', 'Yes.', 'Mouthparts.', 'Spherical.', 'Yes.', 'Yes.', 'Yes.', 'Yes.', 'Yes.', 'Yes.', 'Yes.', 'Yes.']
Batch 25] Time for generation = 5.81 seconds
Batch 26] Started generation...
['<s>[INST] Give an answer to the question. Use one word if possible.\nIn 1881, Grant purchased a house in New York City and placed almost all of his financial assets into an investment banking partnership with Ferdinand Ward, as suggested by Grant\'s son Buck (Ulysses, Jr.), who was having success on Wall Street. Ward swindled Grant (and other investors who had been encouraged by Grant) in 1884, bankrupted the company, Grant & Ward, and fled.\nOn August 22, 1848, Grant married Julia

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


['Julia', 'Point Pleasant.', 'Republican.', 'Grant said "Damn, I had nothing to do with this battle." in response to being asked about his involvement in the Battle of Appomattox Court House.', 'No, Horace Greely did not lose in the presidential elections of 1872. He ran as the candidate of the National Liberty Party and received 48,289 votes in the general election. While he did not win the presidency, he did receive more votes than the candidate of the', 'The given text does not provide any information about why Grant thought the war against Mexico was unjust.', 'Ulysses S. Grant', '1868', 'Grant', '1856.', 'Ulysses S. Grant', 'Yes.', 'Pennsylvania is not mentioned in the given answers.', 'No.', 'Yes.', 'No.', 'Yes.', 'Yes.', 'Lincoln delivered his second inaugural address and the American Civil War ended.', 'Improvement.', 'No.', 'Yes.', 'No.', 'Spanish.', 'San Carlos.', 'Europeans', 'Cantegriles.', 'No.', 'Catholic.', 'Yes.', 'Yes.', 'Yes.']
Batch 26] Time for generation = 9.22 sec

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


['176,214.', '88%', 'Founded.', '2', 'The information provided does not give a direct comparison between the average income of an African woman and a European man.', 'Yes.', 'No.', 'No.', 'Plebiscite.', 'Catholicism, Protestantism, Judaism.', 'Uruguay has been competitive in agriculture by focusing on ecological and natural products, low inputs of labor, technology, and capital, and developing export-oriented agricultural and industrial sectors.', "Uruguay's political parties are: Partido Colorado, Partido Blanco, Frente Amplio.", 'No.', 'Montevideo.', 'Language.', 'Trade.', 'Independence.', 'No.', 'Yes.', 'Yes.', 'No.', 'Yes', 'Yes.', 'Yes.', 'Yes.', 'No.', 'Yes.', 'No.', 'Paris.', '1856', 'Fillmore.', 'History and political science.']
Batch 27] Time for generation = 5.88 seconds
Batch 28] Started generation...
['<s>[INST] Give an answer to the question. Use one word if possible.\nWhen Germany resumed unrestricted submarine warfare in early 1917 and made a clumsy attempt to enlist Mex

## Evaluation

In [26]:
def evaluate_bleu(df_true: pd.DataFrame, df_pred: pd.DataFrame, bleu_type: int):
    df_merged = pd.merge(df_true[['id', 'answer']], df_pred[['id', 'answer']], on='id', suffixes=('_true', '_pred'))

    weights = (1.0, 0.0, 0.0, 0.0) if bleu_type == 1 else (0.5, 0.5, 0.0, 0.0)
    
    bleu_scores = []

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        for _, row in df_merged.iterrows():
            reference = row['answer_true'].split()
            candidate = row['answer_pred'].split() 

            bleu_score = sentence_bleu([reference], candidate, weights=weights)
            bleu_scores.append(bleu_score)
        
    mean_bleu = sum(bleu_scores) / len(bleu_scores)
    return mean_bleu

In [27]:
questions_val = questions_split["validation"].to_pandas()

df_true_faiss = questions_val[['id', 'answer']].copy()
df_pred_faiss = questions_val[['id']].copy()
df_pred_faiss['answer'] = generated_responses_fusion

bleu_1 = evaluate_bleu(df_true_faiss, df_pred_faiss, 1)
bleu_2 = evaluate_bleu(df_true_faiss, df_pred_faiss, 2)

print(f"Le score BLEU-1 est {bleu_1:.3f}.")
print(f"Le score BLEU-2 est {bleu_2:.3f}.\n")

Le score BLEU-1 est 0.158.
Le score BLEU-2 est 0.037.



In [28]:
questions_val = questions_split["validation"].to_pandas()
df_export = questions_val[['id', 'question', 'answer']].copy()
df_export['generated_answer'] = generated_responses_fusion
df_export.to_csv(f"generated_responses_simple_Fusion.csv", index=False, encoding='utf-8')

In [ ]:
from google.colab import files
files.download("generated_responses_simple_Fusion.csv")